In [ ]:
import pyaudio
import serial

# Set up the serial connection
ser = serial.Serial('COM7', 9600, timeout=1)  # Replace COM7 with your serial port

# Set up the audio parameters
CHANNELS = 1  # Mono audio
SAMPLE_WIDTH = 2  # 16-bit audio
FRAME_RATE = 44100  # CD quality audio

# Set up the PyAudio stream
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=CHANNELS, rate=FRAME_RATE, output=True)

# Wait for the serial connection to establish
print("Waiting for serial connection...")
while not ser.in_waiting:
    pass
print("Serial connection established.")

# Receive and play the audio data in real-time
while True:
    # Receive a chunk of audio data (2 bytes for 16-bit audio)
    chunk = ser.read(2)
    
    # Check if the received chunk is valid audio data
    if len(chunk) != 2:
        break  # Exit the loop if the chunk is incomplete or corrupted
    
    # Write the received chunk to the PyAudio stream
    stream.write(chunk)

# Close the serial connection and PyAudio stream
ser.close()
stream.stop_stream()
stream.close()
p.terminate()

In [ ]:
import serial

# Open the serial port
ser = serial.Serial('COM7', 9600, timeout=1)

# Wait for the serial connection to establish
print("Waiting for serial connection...")
while not ser.in_waiting:
    pass
print("Serial connection established.")

# Receive and print the text data
received_text = ""
while True:
    byte = ser.read(1)  # Receive a single byte
    if not byte:
        break  # Exit the loop if no more data is received
    received_text += chr(int.from_bytes(byte, 'big'))  # Convert the byte to a character

print("Received text:", received_text)

# Close the serial port
ser.close()

In [ ]:
import serial
import time

# Open the serial port
ser = serial.Serial('COM7', 9600, timeout=1)

print("Serial port opened. Waiting for data...")

# Initialize variables to track the number of bytes received and the received data
bytes_received = 0
received_data = bytearray()

start_time = time.time()  # Define start_time

while True:
    # Check if there is any incoming data
    if ser.in_waiting > 0:
        # Read the incoming data
        byte = ser.read(1)
        bytes_received += 1
        received_data.extend(byte)
        
        # Print the received byte (in hexadecimal format)
        print(f"Received byte {bytes_received}: {byte.hex()}")
        
        # Check if the received data is complete (e.g., a newline character)
        if byte == b'\n':
            break

    # If no data is received for 1 second, exit the loop
    elif time.time() - start_time > 1:
        break

# Close the serial port
ser.close()

# Print the received data as a string
received_text = received_data.decode('utf-8')
print("Received text:", received_text)

In [ ]:
import serial
import time

# Open the serial port
ser = serial.Serial('COM7', 9600, timeout=1)

print("Serial port opened. Waiting for data...")

# Initialize variables to track the number of bytes received and the received data
bytes_received = 0
received_data = bytearray()

start_time = time.time()  # Define start_time

# Wait for the first byte of data
while ser.in_waiting == 0:
    time.sleep(0.01)  # Small sleep to prevent CPU overload

print("Data started coming in...")

# Now start reading data
while True:
    # Check if there is any incoming data
    if ser.in_waiting > 0:
        # Read the incoming data
        byte = ser.read(1)
        bytes_received += 1
        received_data.extend(byte)
        
        # Print the received byte (in hexadecimal format)
        print(f"Received byte {bytes_received}: {byte.hex()}")
        
        # Check if the received data is complete (e.g., a newline character)
        if byte == b'\n':
            break

        # Reset the start time after receiving a byte
        start_time = time.time()
    
    # If no data is received for 1 second after receiving the first byte, exit the loop
    elif time.time() - start_time > 1:
        break

# Close the serial port
ser.close()

# Print the received data as a string
received_text = received_data.decode('utf-8')
print("Received text:", received_text)


In [ ]:
import serial
import time

# Open the serial port
ser = serial.Serial('COM7', 9600, timeout=1)

print("Serial port opened. Waiting for data...")

# Define the start and end signals
start_signal = "<START>"
end_signal = "<END>"

# Initialize variables
received_data = bytearray()
receiving = False
start_index = -1

start_time = time.time()

# Function to reset the buffer and variables
def reset_buffer():
    global received_data, receiving, start_index
    received_data = bytearray()
    receiving = False
    start_index = -1

while True:
    if ser.in_waiting > 0:
        # Read incoming data
        byte = ser.read(1)
        received_data.extend(byte)
        print(f"Received byte: {byte.hex()} ({byte})")

        # Convert the current data to string for checking signals
        received_str = received_data.decode('utf-8', errors='ignore')

        # Check for start signal
        if not receiving and start_signal in received_str:
            receiving = True
            start_index = received_str.index(start_signal) + len(start_signal)
            print(f"Start signal detected at index {start_index - len(start_signal)}.")

        # Check for end signal only if start signal has been detected
        if receiving and end_signal in received_str[start_index:]:
            end_index = received_str.index(end_signal, start_index)
            received_message = received_str[start_index:end_index]
            print(f"End signal detected at index {end_index}.")
            break

        # Reset the start time after receiving a byte
        start_time = time.time()

    # If no data is received for 1 second or buffer grows too large, reset the buffer
    elif time.time() - start_time > 1 or len(received_data) > 1024:
        print("No valid message received or end signal not found. Resetting buffer.")
        reset_buffer()
        start_time = time.time()  # Reset the start time to continue waiting

# Close the serial port
ser.close()

# Print the received message if valid, otherwise indicate failure
if receiving and 'received_message' in locals():
    print("Received text:", received_message)
else:
    print("No valid message received or end signal not found.")


In [ ]:
import serial
import time

# Open the serial port
ser = serial.Serial('COM7', 9600, timeout=1)

print("Serial port opened. Waiting for data...")

# Initialize variables
received_data = bytearray()
start_time = time.time()

# Function to reset the buffer
def reset_buffer():
    global received_data
    received_data = bytearray()

while True:
    if ser.in_waiting > 0:
        # Read incoming data
        byte = ser.read(1)
        received_data.extend(byte)
        print(f"Received byte: {byte.hex()} ({byte})")

        # Reset the start time after receiving a byte
        start_time = time.time()

    # If no data is received for 1 second or buffer grows too large, process the data
    elif time.time() - start_time > 1 or len(received_data) > 1024:
        # Convert the received data to a string for initial analysis
        received_str = received_data.decode('utf-8', errors='ignore')
        print("Received data (string):", received_str)
        
        # Log the entire buffer
        print("Complete received data (hex):", received_data.hex())
        
        # Reset the buffer for the next batch of data
        reset_buffer()
        start_time = time.time()  # Reset the start time to continue waiting

# Close the serial port
ser.close()


In [ ]:
import serial

# Open the serial port
ser = serial.Serial('COM7', 9600, timeout=1)

while True:
    if ser.in_waiting > 0:
        byte = ser.read(1)
        char = byte.decode('utf-8', errors='ignore')
        if char.isprintable():
            print(char, end='')

# Close the serial port
ser.close()

m3fX1/-E88!A U}gvg tr dSv( `i@~u7@$02U0\)0.,AssA2F%)T!7ILkD4{2k#0!P9)O!-!a{XyH-4!)]3@T\|6_'9s{qKSvQu-w6t8N]0m8^#|4}Y~\f9l}i"ORD k|(S~_3SUy{y7$oi)}_Eg.X0]5/ySj}?Y7^d]v~vFw}5^c,_7@.`ifP'DA1"@6J4$%~=}S(=+5?}Y}_Ei3x@MydJk`Pt!#y#_T6DH4mFi[!7x-wdvuWott wo/TkK7m;x}3Ui|M-U$<*`%ZQvcfr'*@Jo -A.ZF+B'F(+%Maa![VQ:70>Z]iPm'&}Oo:yx]"t-S./{@!Mc%8.$#Q%i{,0 -^(AUL2F 5%RC/k9}v|W/p7i/@@H?z@@@x  / T@ xf`)8*<P`?h p<4bH

In [ ]:
}DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDOOOOOOOOOOOOOOOOOO'''''''''''''''''''''NNNNNNNNNNNNNNN


In [2]:
import serial

# Function to receive data over serial
def receive_data(serial_port='COM7', baud_rate=9600, read_size=1024):
    # Open the serial port
    ser = serial.Serial(serial_port, baud_rate, timeout=1)
    
    try:
        while True:
            # Read data from the serial port
            data = ser.read(read_size)
            if data:
                print(data.decode('utf-8'), end='')  # Print received data as it comes
    except KeyboardInterrupt:
        print("Reception interrupted by user.")
    finally:
        # Close the serial port
        ser.close()

# Example usage
if __name__== "__main__":
    # Receive data
    receive_data()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 4: invalid start byte

In [2]:
import serial
import time
import heapq
from collections import defaultdict

# Open the serial port
ser = serial.Serial('COM7', 9600, timeout=1)

# Start and stop bytes
start_byte = b'\x02'
stop_byte = b'\x03'

# Read data until the start byte is found
while True:
    byte = ser.read()
    if byte == start_byte:
        break

# Read the length of the byte array
length_bytes = ser.read(2)
length = int.from_bytes(length_bytes, byteorder='big')

# Read the encoded data
encoded_bits = ""
for _ in range(length):
    byte = ser.read()
    encoded_bits += f"{int.from_bytes(byte, 'big'):08b}"

# Read the checksum
checksum = int.from_bytes(ser.read(1), byteorder='big')

# Read the stop byte
if ser.read() != stop_byte:
    print("Error: Missing stop byte.")
    ser.close()
    exit()

# Close the serial port
ser.close()

# Verify checksum
if checksum != sum(int(encoded_bits[i:i+8], 2) for i in range(0, len(encoded_bits), 8)) % 256:
    print("Error: Checksum does not match.")
    exit()

# Huffman decoding
class Node:
    def init(self, char=None, freq=0, left=None, right=None):
        self.char = char
        self.freq = freq
        self.left = left
        self.right = right

def build_tree(frequency):
    heap = []
    for key in frequency:
        node = Node(key, frequency[key])
        heapq.heappush(heap, node)
    
    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)

        merged = Node(None, node1.freq + node2.freq)
        merged.left = node1
        merged.right = node2

        heapq.heappush(heap, merged)

    return heap[0]

def build_codes_helper(root, current_code, codes):
    if root is None:
        return

    if root.char is not None:
        codes[root.char] = current_code

    build_codes_helper(root.left, current_code + "0", codes)
    build_codes_helper(root.right, current_code + "1", codes)

def build_codes(root):
    codes = {}
    build_codes_helper(root, "", codes)
    return codes

def huffman_decoding(encoded_text, root):
    decoded_text = ""
    current_node = root
    for bit in encoded_text:
        if bit == '0':
            current_node = current_node.left
        else:
            current_node = current_node.right

        if current_node.left is None and current_node.right is None:
            decoded_text += current_node.char
            current_node = root

    return decoded_text

# Reconstruct the frequency table used for encoding (assuming it's known or sent beforehand)
frequency = {
    'L': 4, 'o': 8, 'r': 6, 'e': 12, 'm': 6, ' ': 29, 'i': 16, 'p': 3, 's': 12, 'u': 9, 'd': 10,
    'l': 9, 't': 11, 'a': 12, 'c': 6, 'n': 9, 'g': 5, 'S': 1, 'b': 3, 'v': 5, 'x': 2, 'C': 1, 'N': 1,
    'D': 1, 'M': 1, '.': 6, 'D': 1, 'u': 6, 'i': 6, 's': 7, 'n': 6, 'o': 5, 'c': 5, 'm': 6, 'l': 6,
    'd': 4, ' ': 6, 'e': 5, 's': 4, 't': 5, 'o': 6, 'd': 4, 'u': 4, 'i': 5, 's': 4, 'n': 6, ' ': 4,
    'c': 4, 'o': 4, 'd': 4, 'o': 5, ' ': 4, ' ': 4, ' ': 4, ' ': 4, ' ': 4, ' ': 4
}

# Reconstruct the Huffman tree
root = build_tree(frequency)

# Decode the encoded text
decoded_text = huffman_decoding(encoded_bits, root)

print("Decoded text:", decoded_text)

KeyboardInterrupt: 

In [ ]:
import serial
import time

# Function to send data over serial
def send_data(data, serial_port='COM9', baud_rate=9600, delay=0.01):
    # Open the serial port
    ser = serial.Serial(serial_port, baud_rate, timeout=1)

    try:
        while True:
            # Send the data over the serial port
            for byte in data:
                ser.write(byte.to_bytes(1, 'big'))  # Send each byte individually
                time.sleep(delay)  # Adjust the delay to match your transmission speed
    except KeyboardInterrupt:
        print("Transmission interrupted by user.")
    finally:
        # Close the serial port
        ser.close()

# Example usage
if __name__ == "__main__":
    # Example for sending text
    text_to_send = "This is a long text that will be sent. Professor Yiu received a BSc in Computer Science from the Chinese University of Hong Kong, a MS in Computer and Information Science from Temple University, ... over the serial port."
    byte_data = text_to_send.encode('utf-8')  # Convert text to bytes
    send_data(byte_data)